<a href="https://colab.research.google.com/github/saravanamuthu1/kryol_language_segementation/blob/main/Kryol_segementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We use bidirectional LSTM from the previous task

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
from tensorflow.keras.metrics import CategoricalAccuracy
from tqdm import tqdm
import csv

getting the training data from the train.tsv


In [2]:
data = open('train.tsv', 'r')
all_words = []
all_segemented_words = []
for line in data:
    pieces = line.rstrip("\n").split("\t")
    all_words.append(pieces[0])
    all_segemented_words.append(pieces[1])
    

test data from the test.txt


In [3]:
with open('test.txt') as f: 
  test_data=[]
  for line in f: 
    value= line.rstrip("\n")
    test_data.append(value)

In [4]:
validation_data=[]

In [5]:
with open('test.txt') as f: 
  for line in f: 
    line=line.rstrip('\n')
    validation_data.append(line)

labelling the train data as B and I


In [6]:
complete_list=[]
all_label = []
for i in all_segemented_words:
  value=i.split("-")
  cur_word = []
  word_label=[]
  for i in range(0, len(value)):
        for j in range(0, len(value[i])):
            if j == 0:
                cur_word.append(value[i][j])
                word_label.append("B")
            else:
                cur_word.append(value[i][j])
                word_label.append("I")
  complete_list.append(cur_word)
  all_label.append(word_label)


Finding the longest string


In [7]:
longest_string = max(complete_list, key=len)
print(len(longest_string))

18


padding length

In [8]:
max_len = 20

In [9]:
len_letters= len(set(letter for word in complete_list for letter in word ))

enoding  the test set and training set

In [10]:
letters= list(set(letter for word in complete_list for letter in word ))
letters.append('<PAD>')

In [11]:
test_letters= list(set(letter for word in test_data for letter in word ))
test_letters.append('<PAD>')

In [12]:
len(test_letters)

51

In [13]:
tags_value= len(set(tag for tag_list in all_label for tag in tag_list))

In [14]:
tags_value= list(set(tag for tag_list in all_label for tag in tag_list))

In [15]:
tags_value.append('<PAD>')

In [16]:
word2index = {w: i for i, w in enumerate(letters)}

In [17]:
onehot = [[word2index[w] for w in s] for s in complete_list]

In [18]:
onehot_test= [[word2index[w] for w in s] for s in test_data]

In [19]:
X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(letters)-1)

In [20]:
X_test= pad_sequences(maxlen=max_len, sequences=onehot_test, padding="post", value=len(letters)-1)

In [21]:
from tensorflow.keras.utils import to_categorical

In [22]:
tag2index = {t: i for i, t in enumerate(tags_value)}

In [23]:
onehot_y = [[tag2index[w] for w in s] for s in all_label]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
y = to_categorical(y, num_classes=len(tags_value))

Bidirectional LSTM

In [24]:
model = Sequential()
model.add(Embedding(input_dim=len(letters), output_dim=50, input_length=max_len))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(len(tags_value), activation="softmax")))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            2600      
                                                                 
 bidirectional (Bidirectiona  (None, 20, 200)          120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 20, 3)            603       
 ibuted)                                                         
                                                                 
Total params: 124,003
Trainable params: 124,003
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# run fit on Colab or hopper
history = model.fit(X, y, batch_size=10, epochs=1, validation_split=0.2, verbose=1)

1025/1025 [==============================] - 66s 59ms/step - loss: 0.0551 - accuracy: 0.9807 - val_loss: 0.0067 - val_accuracy: 0.9984


Test data predictions

In [26]:
predicted_test=[]
end_testoutput=[]
for test_sent in range(len(X_test)):
  temp=[]
  pred = model.predict(np.array([X_test[test_sent]]))
  p = np.argmax(pred, axis=-1)
  for i,ix in enumerate(X_test[test_sent]):
    if ix==len(letters)-1:
        break
    temp.append(tags_value[p[0][i]])
    predicted_test.append(tags_value[p[0][i]])
  end_testoutput.append(temp)

Training data prediction

In [27]:
predicted=[]
actual=[]
for test_sent in range(len(X)):
  pred = model.predict(np.array([X[test_sent]]))
  p = np.argmax(pred, axis=-1)
  p_actual = np.argmax(y[test_sent],axis=-1)
  for i,ix in enumerate(X[test_sent]):
    if ix==len(letters)-1:
        break
    predicted.append(tags_value[p[0][i]])
    actual.append(tags_value[p_actual[i]])


Converting the predcitions to its graphemes

In [28]:
string_output=[]
count=0
for i,j in zip(test_data,end_testoutput):
  count=count+1
  empyt_string=[]
  for index in range(0,len(i)):
    if index <len(i)-1:
      if j[index]== "B" and j[index+1] == "B":
        empyt_string.append(i[index])
      elif j[index]== "B" and j[index+1] == "I"and index<len(i)-1:
        value=i[index]+i[index+1]
        empyt_string.append(value)
    elif j[index] == "B":
      empyt_string.append(i[index])
  string_output.append(empyt_string)



In [29]:
string_output1=[]
for i in string_output:
  string_value=""
  for j in range(len(i)):
    if j!=len(i)-1:
      string_value=string_value+i[j]+"-"
    else:
      string_value=string_value+i[j]
  string_output1.append(string_value)



In [30]:
df=pd.DataFrame(predicted_test)

In [31]:
len(string_output)

1427

In [32]:
df.value_counts()

B    8705
I     894
dtype: int64

In [33]:
df1=pd.DataFrame(actual)

In [34]:
df1.value_counts()

B    79696
I     7608
dtype: int64

In [35]:
count=0

In [36]:
total_value=list(zip(predicted,actual))

accuracy for training set

In [37]:
from sklearn.metrics import accuracy_score


In [38]:
accuracy_score(predicted,actual)

0.9942957940071474

Recall Score


In [39]:
true_pos=0
false_neg=0
for i,j in zip(predicted,actual):
   if i == 'I' and j == 'I':
     true_pos += 1
   if i == 'B' and j == 'I':
      false_neg += 1


In [40]:
recall = 100 * (true_pos / (true_pos + false_neg))

Precision Score


In [41]:
true_pos1 = 0
false_pos1 = 0
for i,j in zip(predicted,actual):
   if i == 'I' and j == 'I':
     true_pos1 += 1
   if i == 'I' and j == 'B':
      false_pos1 += 1

In [42]:
precision = 100 * (true_pos1 / (true_pos1 + false_pos1))

In [43]:
precision

95.53022540983606

Fscore

In [44]:
fscore = (2 * (precision * recall)) / (precision + recall)
print("Fscore: ", fscore)

Fscore:  96.7695900363259


Final Graphemes

In [45]:
f = open("output.txt", "a",encoding='utf8')
for i,j in zip(validation_data,string_output1):
  f.write(i)
  f.write("\t")
  f.write(j)
  f.write('\n')
f.close()